<a href="https://colab.research.google.com/github/AIDevLabOrg/chat_ccent_colab/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Change directory to the desired path in Google Drive
%cd /content/drive/MyDrive/Chatccent


Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1K01ZKskFY_3xlmHsN7rkcYdC5TyTsst9/Chatccent


In [2]:
%cd /content/drive/MyDrive/Chatccent

/content/drive/.shortcut-targets-by-id/1K01ZKskFY_3xlmHsN7rkcYdC5TyTsst9/Chatccent


In [3]:
%cd /content/drive/MyDrive/Chatccent/MeloTTS

/content/drive/.shortcut-targets-by-id/1K01ZKskFY_3xlmHsN7rkcYdC5TyTsst9/Chatccent/MeloTTS


In [ ]:
!ls

Dockerfile  LICENSE   melo	      README.md		setup.py
docs	    logo.png  meta_data.list  requirements.txt	test


## Segemt Audio

In [ ]:
# Step 1: Install dependencies
!pip install pydub
!apt-get install -y ffmpeg

# Step 2: Import libraries
from pydub import AudioSegment
import os

# Step 3: Define the function to split the audio
def split_audio_directory(input_dir, segment_duration=9000, sample_rate=44100, output_dir="/content/drive/MyDrive/Chatccent/audios"):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get a list of audio files in the directory
    audio_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp3', '.wav'))]
    total_files = len(audio_files)

    # Initialize counters
    file_counter = 0
    total_segments = 0

    # Loop through all audio files in the directory
    for filename in audio_files:
        file_counter += 1
        file_path = os.path.join(input_dir, filename)

        print(f"Processing file {file_counter}/{total_files}: {filename}")

        # Check file extension and load audio
        if filename.endswith('.mp3'):
            audio = AudioSegment.from_mp3(file_path)
        elif filename.endswith('.wav'):
            audio = AudioSegment.from_wav(file_path)
        else:
            print(f"Unsupported file format for {filename}. Skipping.")
            continue

        # Set the sample rate
        audio = audio.set_frame_rate(sample_rate)

        # Calculate the number of segments
        num_segments = len(audio) // segment_duration
        total_segments += num_segments + 1  # Include the last segment

        # Split the audio into segments and save each one in the output directory
        for i in range(num_segments + 1):
            start_time = i * segment_duration
            end_time = start_time + segment_duration

            # Extract segment
            segment = audio[start_time:end_time]

            # Export segment as .wav into the same directory with a unique name
            segment.export(f"{output_dir}/segment_{file_counter}_{i + 1}.wav", format="wav")
            print(f"Exported segment {i + 1}/{num_segments + 1} for {filename} to {output_dir}/segment_{file_counter}_{i + 1}.wav")

    print(f"Audio splitting complete for all files!")
    print(f"Total audio files processed: {total_files}")
    print(f"Total segments created: {total_segments}")

# Step 4: Specify the directory containing the audio files
input_audio_dir = "/content/drive/MyDrive/Chatccent/Complete_audios"  # Update this to your audio directory path

# Step 5: Run the split function on the specified directory
split_audio_directory(input_audio_dir)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Processing file 1/43: Why_is_the_Colosseum_“broken”__Here’s_the_answer_.wav
Exported segment 1/6 for Why_is_the_Colosseum_“broken”__Here’s_the_answer_.wav to /content/drive/MyDrive/Chatccent/audios/segment_1_1.wav
Exported segment 2/6 for Why_is_the_Colosseum_“broken”__Here’s_the_answer_.wav to /content/drive/MyDrive/Chatccent/audios/segment_1_2.wav
Exported segment 3/6 for Why_is_the_Colosseum_“broken”__Here’s_the_answer_.wav to /content/drive/MyDrive/Chatccent/audios/segment_1_3.wav
Exported segment 4/6 for Why_is_the_Colosseum_“broken”__Here’s_the_answer_.wav to /content/drive/MyDrive/Chatccent/audios/segment_1_4.wav
Exported segment 5/6 for Why_is_the_Colosseum_“broken”__Here’s_the_answer_.wav to /content/drive/MyDrive/Chatccent/audios/segment_1_5.wav
Exported

## Remove Silence

In [4]:
!pip install pydub

## Meta data

In [5]:
!pip install openai-whisper
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=f07b5de8b31e25615ef23241662794b512d6d154103eb72c642e089664cd5701
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [ ]:
# Step 2: Import necessary libraries
import os
import whisper

# Step 3: Define the transcription and meta list generation function
def generate_meta_list(audio_dir, speaker_name="EN-default ", language_code="EN", output_file="meta_data.list"):
    # Load the Whisper model for transcription
    model = whisper.load_model("large")  # Use 'base' model, or choose 'tiny', 'small', 'medium', 'large' as per your needs

    # Open the output file in write mode
    with open(output_file, "w") as f:
        for idx, file_name in enumerate(sorted(os.listdir(audio_dir))):
            if file_name.endswith(".wav"):
                # Get the full path of the audio file
                file_path = os.path.join(audio_dir, file_name)

                # Get the relative path (remove '/content/drive/MyDrive/')
                relative_path = os.path.relpath(file_path, start="/content/drive/MyDrive/Chatccent/MeloTTS/melo/")

                # Transcribe the audio
                transcription = model.transcribe(file_path)
                text = transcription["text"]

                # Format the output as required
                meta_entry = f"{relative_path}|{speaker_name}|{language_code}|{text}\n"

                # Write the formatted entry to the file
                f.write(meta_entry)
                print(f"Processed and added: {relative_path}")

    print(f"Meta list generated and saved to {output_file}")

# Step 4: Run the function
audio_directory = "/content/drive/MyDrive/Chatccent/MeloTTS/melo/data/example/audios"  # Replace this with your audio directory
generate_meta_list(audio_directory)


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Processed and added: data/example/audios/segment_10_1.wav
Processed and added: data/example/audios/segment_10_10.wav
Processed and added: data/example/audios/segment_10_11.wav
Processed and added: data/example/audios/segment_10_12.wav
Processed and added: data/example/audios/segment_10_13.wav
Processed and added: data/example/audios/segment_10_14.wav
Processed and added: data/example/audios/segment_10_15.wav
Processed and added: data/example/audios/segment_10_16.wav
Processed and added: data/example/audios/segment_10_17.wav
Processed and added: data/example/audios/segment_10_18.wav
Processed and added: data/example/audios/segment_10_19.wav
Processed and added: data/example/audios/segment_10_2.wav
Processed and added: data/example/audios/segment_10_20.wav
Processed and added: data/example/audios/segment_10_21.wav
Processed and added: data/example/audios/segment_10_22.wav
Processed and added: data/example/audios/segment_10_23.wav
Processed and added: data/example/audios/segment_10_24.wav

In [6]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 47.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 98.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━

In [ ]:
# Sample input lines as you described
lines = [
    "/content/drive/MyDrive/Chatccent/audios/segment_1.wav|IT-default|IT| You must have noticed the Colosseum is, let's say, broken. In fact, only one third of its original size is...",
    "/content/drive/MyDrive/Chatccent/audios/segment_2.wav|IT-default|IT| still standing today. How incredible is that? The reason why there's so much missing of the Colosseum is due to a variety of reasons.",
    "/content/drive/MyDrive/Chatccent/audios/segment_3.wav|IT-default|IT| For sure, earthquakes, abandonments, old buildings, but most of all, the pieces of the Colosseum were taken.",
    "/content/drive/MyDrive/Chatccent/audios/segment_4.wav|IT-default|IT| off, were stripped off the Colosseum to build new things. I know it sounds hard to believe but that's what happened. Since the Middle Ages,",
    "/content/drive/MyDrive/Chatccent/audios/segment_5.wav|IT-default|IT| And for centuries, the popes, the richest families of Rome, they took pieces of the Colosseum to build palaces, churches.",
    "/content/drive/MyDrive/Chatccent/audios/segment_6.wav|IT-default|IT| We even know which of these buildings are made with the stone of the Colosseum. For example, Palazzo Barberini, the church of the Pope.",
    "/content/drive/MyDrive/Chatccent/audios/segment_7.wav|IT-default|IT| to San Paolo Stino, and many other things. How incredible is that? And you, have you ever seen the Colosseum?",
    "/content/drive/MyDrive/Chatccent/audios/segment_8.wav|IT-default|IT| when walking around the city? Tell us in the comments!",
]

# Desired output
formatted_lines = []

for line in lines:
    # Split the line by '|'
    parts = line.split('|')
    if len(parts) >= 4:
        # Extract the relevant parts
        audio_path = parts[0].strip()
        speaker_name = parts[1].strip().split('-')[0]  # Assuming speaker name is before '-'
        language_code = parts[2].strip()
        text = '|'.join(parts[3:]).strip()  # Join the rest back as text

        # Format the line
        formatted_line = f"{audio_path} | {speaker_name} | {language_code} | {text}"
        formatted_lines.append(formatted_line)

# Save the formatted lines to a file
output_file_path = '/content/drive/MyDrive/Chatccent/meta_list.txt'
with open(output_file_path, 'w') as f:
    for formatted_line in formatted_lines:
        f.write(formatted_line + '\n')

print(f"Formatted lines saved to {output_file_path}")


Formatted lines saved to /content/drive/MyDrive/Chatccent/meta_list.txt


In [7]:
%cd /content/drive/MyDrive/Chatccent/MeloTTS/melo

/content/drive/.shortcut-targets-by-id/1K01ZKskFY_3xlmHsN7rkcYdC5TyTsst9/Chatccent/MeloTTS/melo


In [8]:
!pip install unidic-lite
!python -m unidic download


download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:26<00:00, 19.7MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.10/dist-packages/unidic/dicdir


In [9]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [10]:
import MeCab
tagger = MeCab.Tagger("-Owakati")
print(tagger.parse("これはテストです。"))


これ は テスト です 。 



In [ ]:
!python preprocess_text.py --metadata /content/drive/MyDrive/Chatccent/MeloTTS/melo/data/example/metadata.list

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  0% 0/2202 [00:00<?, ?it/s]Processing line: data/example/audios/segment_10_1.wav|EN-default |EN| The biggest imperial palace of ancient Rome, an ancient collection of travel souvenirs, an incredible day trip from Rome. What's this place?
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when pos

In [ ]:
rank = 0  # Since only one GPU is available
torch.cuda.set_device(rank)


In [ ]:
!bash train.sh /content/drive/MyDrive/Chatccent/MeloTTS/melo/data/example/config.json 1

2024-12-26 14:05:17.765572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-26 14:05:17.785699: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-26 14:05:17.791730: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-26 14:05:17.807033: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-26 14:05:19.281249: W tensorflow/compiler/tf2